# Import and Clean Data

The data must be imported from a CSV file, cleaned to the standards of the question being asked, and adapted into a neural network input.


## References

 * [Andrew Long](https://github.com/andrewwlong/diabetes_readmission/blob/master/diabetes_project.ipynb)
 * [Sheng Weng](https://github.com/swengzju/Predicting-Diabetes-Patient-Readmission/blob/master/Predicting%20Diabetes%20Patient%20Readmission.ipynb)
 * [Bose, et al.](https://github.com/Yawhoong/ISS-Diabetes-Readmission/blob/master/Identification%20of%20Critical%20Risk%20Factors%20leading%20to%20short-term%20readmission%20of%20Diabetic%20Patients.pdf)


In [1]:
# Import modules and data
import pandas as pd
import numpy as np

df = pd.read_csv("diabetic_data.csv")
df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [2]:
# Determine whether the diabetes diagnosis was in the top 3
# Drop the diagnoses, they are too scattered
import re
DIABETES_REGEX = re.compile("250")
isDiabetes = lambda s: DIABETES_REGEX.match(s)
df["primary_diag"] = (df["diag_1"].str.contains("^250") |
                      df["diag_2"].str.contains("^250") |
                      df["diag_3"].str.contains("^250")).astype("int")
df.drop(["diag_1", "diag_2", "diag_3"], axis=1, inplace=True)
df.groupby("primary_diag").size()

primary_diag
0    63742
1    38024
dtype: int64

In [3]:
# Remove columns...
#  * encounter_id, patient_nbr (identification)
#  * weight, payer_code, medical_specialty (too sparse)
#  * diag_1, diag_2, diag_3 (too specific)
df.drop(["encounter_id", "patient_nbr", "weight", "payer_code", "medical_specialty"], axis=1, inplace=True)
df.head()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,...,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,primary_diag
0,Caucasian,Female,[0-10),6,25,1,1,41,0,1,...,No,No,No,No,No,No,No,No,NO,1
1,Caucasian,Female,[10-20),1,1,7,3,59,0,18,...,Up,No,No,No,No,No,Ch,Yes,>30,1
2,AfricanAmerican,Female,[20-30),1,1,7,2,11,5,13,...,No,No,No,No,No,No,No,Yes,NO,1
3,Caucasian,Male,[30-40),1,1,7,2,44,1,16,...,Up,No,No,No,No,No,Ch,Yes,NO,1
4,Caucasian,Male,[40-50),1,1,7,1,51,0,8,...,Steady,No,No,No,No,No,Ch,Yes,NO,1


In [4]:
# Filter by...
#  * Must have a gender
df = df[df["gender"] != "Unknown/Invalid"]

#  * Must not be discharged by death or hospice
df = df.loc[~df["discharge_disposition_id"].isin([11, 13, 14, 19, 20, 21])]

df.shape

(99340, 43)

In [5]:
# Normalize continuous columns to [0, 1]
cols_continuous = ['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient','number_diagnoses']

for col in cols_continuous:
    lo, hi = df[col].min(), df[col].max()
    df[col] = (df[col] - lo) / (hi - lo)

In [6]:
# Normalize numerically-categorical columns (i.e. age)
AGE_MAPPING = {"[{}-{})".format(i * 10, (i + 1) * 10): i * 0.1 + 0.05 for i in range(10)}
df["age"] = df["age"].apply(lambda s: AGE_MAPPING[s])
df[["age"] + cols_continuous].head()

,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
0,0.05,0.000000,0.305344,0.000000,0.0000,0.000000,0.0,0.000000,0.000000
1,0.15,0.153846,0.442748,0.000000,0.2125,0.000000,0.0,0.000000,0.533333
2,0.25,0.076923,0.076336,0.833333,0.1500,0.047619,0.0,0.047619,0.333333
3,0.35,0.076923,0.328244,0.166667,0.1875,0.000000,0.0,0.000000,0.400000
4,0.45,0.000000,0.381679,0.000000,0.0875,0.000000,0.0,0.000000,0.266667


In [7]:
# Manipulate categorical columns
cols_categorical = ['race', 'gender',
        'admission_type', 'admission_source',
        'max_glu_serum', 'A1Cresult',
        'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
        'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
        'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
        'tolazamide', 'insulin',
        'glyburide-metformin', 'glipizide-metformin',
        'glimepiride-pioglitazone', 'metformin-rosiglitazone',
        'metformin-pioglitazone', 'change', 'diabetesMed',
        'examide', 'citoglipton']

# Create an "Unknown" category for race
df["race"] = df["race"].replace("?", "Unknown")

# Consolidate admissions types
ADMISSION_TYPE_MAPPING = {
    1: "Emergency",
    2: "Urgent",
    3: "Elective"
}
df["admission_type_id"] = df["admission_type_id"].apply(lambda s: ADMISSION_TYPE_MAPPING.get(s, "Other"))

# Discharge disposition is either going home OK or not going home OK
df["discharge_disposition_id"] = (df["discharge_disposition_id"] == 1).astype("int")

# Admission source is either Emergency, Referral, or Other
ADMISSION_SOURCE_MAPPING = {
    1: "EmergencyRoom",
    7: "Referral"
}
df["admission_source_id"] = df["admission_type_id"].apply(lambda s: ADMISSION_SOURCE_MAPPING.get(s, "Other"))

df.rename(columns={
    "admission_type_id": "admission_type",
    "discharge_disposition_id": "discharged_home",
    "admission_source_id": "admission_source"}, inplace=True)
#df["race", "admission_type", "admission_source", "discharged_home"].head()
df.head()

,race,gender,age,admission_type,discharged_home,admission_source,time_in_hospital,num_lab_procedures,num_procedures,num_medications,...,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,primary_diag
0,Caucasian,Female,0.05,Other,0,Other,0.000000,0.305344,0.000000,0.0000,...,No,No,No,No,No,No,No,No,NO,1
1,Caucasian,Female,0.15,Emergency,1,Other,0.153846,0.442748,0.000000,0.2125,...,Up,No,No,No,No,No,Ch,Yes,>30,1
2,AfricanAmerican,Female,0.25,Emergency,1,Other,0.076923,0.076336,0.833333,0.1500,...,No,No,No,No,No,No,No,Yes,NO,1
3,Caucasian,Male,0.35,Emergency,1,Other,0.076923,0.328244,0.166667,0.1875,...,Up,No,No,No,No,No,Ch,Yes,NO,1
4,Caucasian,Male,0.45,Emergency,1,Other,0.000000,0.381679,0.000000,0.0875,...,Steady,No,No,No,No,No,Ch,Yes,NO,1


In [8]:
# Handle categorical columns...
#  1. Remove categorical columns that are almost uniform (>99% of entries are one thing)
#  2. Split other columns into multiple boolean columns
n_rows = df.shape[0]
cols_insignificant = []
cols_significant = []
for col in cols_categorical:
    freqs = dict(df.groupby(col).size())
    
    # Remove this column
    if any([float(value)/float(n_rows) > 0.99 for value in freqs.values()]):
        cols_insignificant.append(col)
    
    else:
        # Sanity check
        assert(len(freqs) > 1)
        cols_significant.append(col)
        
        # Convert to column to string if not already string type
        if df[col].dtype != str:
            df[col] = df[col].astype(str)
            
# Extract output
df_output = pd.get_dummies(df["readmitted"])
df.drop(["readmitted"], axis=1, inplace=True)

df_cat = pd.get_dummies(df[cols_significant], drop_first=False)
df_input = pd.concat([df, df_cat], axis=1)

df_input.drop(cols_insignificant + cols_significant, axis=1, inplace=True)

print("Insignificant: {}".format(cols_insignificant))
print("Significant: {}".format(cols_significant))
df_input.head()

Insignificant: ['admission_source', 'nateglinide', 'chlorpropamide', 'acetohexamide', 'tolbutamide', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 'glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone', 'examide', 'citoglipton']
Significant: ['race', 'gender', 'admission_type', 'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'glimepiride', 'glipizide', 'glyburide', 'pioglitazone', 'rosiglitazone', 'insulin', 'change', 'diabetesMed']


,age,discharged_home,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,...,rosiglitazone_Steady,rosiglitazone_Up,insulin_Down,insulin_No,insulin_Steady,insulin_Up,change_Ch,change_No,diabetesMed_No,diabetesMed_Yes
0,0.05,0,0.000000,0.305344,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,...,0,0,0,1,0,0,0,1,1,0
1,0.15,1,0.153846,0.442748,0.000000,0.2125,0.000000,0.0,0.000000,0.533333,...,0,0,0,0,0,1,1,0,0,1
2,0.25,1,0.076923,0.076336,0.833333,0.1500,0.047619,0.0,0.047619,0.333333,...,0,0,0,1,0,0,0,1,0,1
3,0.35,1,0.076923,0.328244,0.166667,0.1875,0.000000,0.0,0.000000,0.400000,...,0,0,0,0,0,1,1,0,0,1
4,0.45,1,0.000000,0.381679,0.000000,0.0875,0.000000,0.0,0.000000,0.266667,...,0,0,0,0,1,0,1,0,0,1


At this point, all cells in the data should be numeric between 0 and 1. There are three different outcomes represented. Outputs are left in the same table as inputs to ensure that they do not get scrambled.

In [9]:
df = pd.concat([df_input, df_output], axis=1)
df.rename(columns={"<30": "OUTPUT_<30", ">30": "OUTPUT_>30", "NO": "OUTPUT_NO"}, inplace=True)
df.head()

,age,discharged_home,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,...,insulin_No,insulin_Steady,insulin_Up,change_Ch,change_No,diabetesMed_No,diabetesMed_Yes,OUTPUT_<30,OUTPUT_>30,OUTPUT_NO
0,0.05,0,0.000000,0.305344,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,...,1,0,0,0,1,1,0,0,0,1
1,0.15,1,0.153846,0.442748,0.000000,0.2125,0.000000,0.0,0.000000,0.533333,...,0,0,1,1,0,0,1,0,1,0
2,0.25,1,0.076923,0.076336,0.833333,0.1500,0.047619,0.0,0.047619,0.333333,...,1,0,0,0,1,0,1,0,0,1
3,0.35,1,0.076923,0.328244,0.166667,0.1875,0.000000,0.0,0.000000,0.400000,...,0,0,1,1,0,0,1,0,0,1
4,0.45,1,0.000000,0.381679,0.000000,0.0875,0.000000,0.0,0.000000,0.266667,...,0,1,0,1,0,0,1,0,0,1


# Neural Network

Bose, et al. used a neural network with the following parameters to model the given data:

 * Layers:
    1. Input (size = 70)
    2. Hidden Layer 1 (size = 70, ReLU activation)
    3. Hidden Layer 2 (size = 20, ReLU activation)
    4. Output Layer (size = 2, softmax)  
 * Dropout Rate: 0.1
 * Batch Size: 100
 * Epoch Size: 50
 * Optimization Algorithm: Adaptive Moment Estimation
 * Cost Function: Optimizes for recall
 
[This tutorial](https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/multilayer_perceptron.py) is useful for implementing a basic multi-layer perceptron classifier.

In [10]:
import tensorflow as tf
import matplotlib.pyplot as plt

# Hyperparameters
N_INPUTS = 67

## Data Separation

The data is separated into training, validation, and testing pools using a random number generator always seeded with the same value (so separation occurs the same way every time)

In [11]:
# Pull out validation (15%) and test (15%) data from training (70%) data
df_valid_test = df.sample(frac=0.3, random_state=0xda)
df_training = df.drop(df_valid_test.index)

df_valid = df_valid_test.sample(frac=0.5, random_state=0xdb)
df_test = df_valid_test.drop(df_valid.index, axis=0)

df_training.shape[0], df_valid.shape[0], df_test.shape[0]

(69538, 14901, 14901)

## Batches

The following class can be used as an iterable in a `for` loop. It takes a dynamic collection of data separated into pools (e.g. categories like readmission vs. no readmission) and produces a batch which samples equally frequently from each pool. The iterable stops when all data in the largest pool have been iterated over once

In [12]:
class TrainingBatches:

    def __init__(self, nn, batch_size=16):
        self.nn = nn
        self.batch_size = batch_size
        self.bounds = max([d.shape[0] for d in nn.training_data])
        self.it = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.it >= self.bounds:
            raise StopIteration

        output = np.concatenate([self.getSlice(d, self.it, self.batch_size) for d in self.nn.training_data])
        self.it += self.batch_size
        return output[:, :N_INPUTS], output[:, N_INPUTS:]

    # Code must work with Python 2 and 3
    next = __next__

    def getSlice(self, array, start, length):
        start = start % array.shape[0]
        if start + length >= array.shape[0]:
            return np.concatenate((array[start:], array[:(start + length) % array.shape[0]]), axis=0)
        else:
            return array[start:start + length]

In [13]:
class CategoricalPreprocessor:
    
    def __init__(self, df, output_categories):
        self.output_categories = output_categories
        self.df = df
        self.is_modified = False
        
    def modifyDatafile(self):
        
        if self.is_modified:
            return self.df
        
        self.df = self.df.copy()
        self.is_modified = True
        
        if self.output_categories == "any":
            self.df["OUTPUT_ANY"] = self.df["OUTPUT_<30"] + self.df["OUTPUT_>30"]
            self.df.drop(["OUTPUT_<30", "OUTPUT_>30"], axis=1, inplace=True)

        elif self.output_categories == "rapid":
            self.df["OUTPUT_NO"] = self.df["OUTPUT_>30"] + self.df["OUTPUT_NO"]
            self.df.drop(["OUTPUT_>30"], axis=1, inplace=True)
            
        return self.df
    
    def getArraysByOutput(self):
        
        if not self.is_modified:
            self.modifyDatafile()
            
        if self.output_categories == "three":
            return (
                self.df[self.df["OUTPUT_<30"] == 1].to_numpy(),
                self.df[self.df["OUTPUT_>30"] == 1].to_numpy(),
                self.df[self.df["OUTPUT_NO"] == 1].to_numpy()
            )

        elif self.output_categories == "any":
            return (
                self.df[self.df["OUTPUT_ANY"] == 1].to_numpy(),
                self.df[self.df["OUTPUT_NO"] == 1].to_numpy()
            )

        else:
            return (
                self.df[self.df["OUTPUT_<30"] == 1].to_numpy(),
                self.df[self.df["OUTPUT_NO"] == 1].to_numpy()
            )

## Neural Network Generator

The following class dynamically generates a neural network with its own training and validation data

In [14]:
class NeuralNetwork:

    def __init__(self, df, df_validation, layer_sizes, output_categories, dropout, batch_size):
        self.dropout = dropout
        self.output_categories = output_categories
        self.batch_size = batch_size
        self.layer_sizes = layer_sizes
        
        # Prepare training data
        training_processor = CategoricalPreprocessor(df, output_categories)
        self.training_data = training_processor.getArraysByOutput()
        
        # Prepare training batches
        training_in_out = list(iter(TrainingBatches(self, 32)))
        self.training_in = np.concatenate([i for i, o in training_in_out])
        self.training_out = np.concatenate([o for i, o in training_in_out])
        
        # Prepare validation data
        validation_processor = CategoricalPreprocessor(df_validation, output_categories)
        self.validation_data = validation_processor.modifyDatafile().to_numpy()
        self.validation_in = self.validation_data[:, :N_INPUTS]
        self.validation_out = self.validation_data[:, N_INPUTS:]
        
        # Create layers
        self.classifier = tf.keras.Sequential()
        self.classifier.add(tf.keras.layers.Dense(
            layer_sizes[0], activation="sigmoid", kernel_initializer="random_normal", input_dim=N_INPUTS))
        self.classifier.add(tf.keras.layers.Dropout(self.dropout))
        for size in layer_sizes[1:]:
            self.classifier.add(tf.keras.layers.Dense(
                size, activation="sigmoid", kernel_initializer="random_normal"))
            self.classifier.add(tf.keras.layers.Dropout(self.dropout))
        self.classifier.add(tf.keras.layers.Dense(
            len(self.training_data), activation="softmax", kernel_initializer="random_normal"))
        
        # Compile model
        self.classifier.compile(optimizer="adam", loss="binary_crossentropy",
                                metrics=["accuracy", tf.keras.metrics.AUC(name="auc")])
        
    def train(self, epochs=20):
        return self.classifier.fit(self.training_in, self.training_out,
                                   validation_data=(self.validation_in, self.validation_out),
                                   batch_size=self.batch_size,
                                   epochs=epochs)
    
    def configStr(self):
        return "Layers: {}; Output Categories: {}; Dropout: {}; Batch Size: {}".format(
            self.layer_sizes, self.output_categories, self.dropout, self.batch_size)

In [15]:
nn = NeuralNetwork(**{
    "df": df_training.copy(),
    "df_validation": df_valid.copy(),
    "layer_sizes": (70, 20),
    "output_categories": "three",
    "dropout": 0.1,
    "batch_size": 64})

W1001 16:21:18.614584 139725043808064 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow/python/keras/initializers.py:143: calling __init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [16]:
hist = nn.train(5)

W1001 16:21:18.862196 139725043808064 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/math_grad.py:1250: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 110208 samples, validate on 14901 samples
Epoch 1/5
110208/110208 [==============================] - 5s 41us/sample - loss: 0.6309 - acc: 0.6671 - auc: 0.5582 - val_loss: 0.6232 - val_acc: 0.6687 - val_auc: 0.5918
Epoch 2/5
110208/110208 [==============================] - 4s 40us/sample - loss: 0.6196 - acc: 0.6708 - auc: 0.6072 - val_loss: 0.6093 - val_acc: 0.6732 - val_auc: 0.6465
Epoch 3/5
110208/110208 [==============================] - 4s 39us/sample - loss: 0.6152 - acc: 0.6730 - auc: 0.6187 - val_loss: 0.6191 - val_acc: 0.6663 - val_auc: 0.6189
Epoch 4/5
110208/110208 [==============================] - 4s 40us/sample - loss: 0.6133 - acc: 0.6737 - auc: 0.6231 - val_loss: 0.6055 - val_acc: 0.6744 - val_auc: 0.6578
Epoch 5/5
110208/110208 [==============================] - 4s 39us/sample - loss: 0.6123 - acc: 0.6739 - auc: 0.6263 - val_loss: 0.6175 - val_acc: 0.6693 - val_auc: 0.6219


In [17]:
hist.history

{'acc': [0.6670509, 0.67084074, 0.6729516, 0.6736687, 0.67393214],
 'auc': [0.5582249, 0.60722375, 0.6187461, 0.62312526, 0.62629104],
 'loss': [0.6309011867237423,
  0.6195571417038725,
  0.6152096582829745,
  0.6133293910308998,
  0.6123258302920648],
 'val_acc': [0.6687022, 0.6732209, 0.6662864, 0.6744064, 0.66926146],
 'val_auc': [0.59177387, 0.6465055, 0.61885697, 0.65775895, 0.62191576],
 'val_loss': [0.6232137545172252,
  0.6092648824767998,
  0.6191400284575469,
  0.6055272697330393,
  0.6174615099801128]}

## Hyper-Parameter Tuning

The hyper-parameters can be tuned to optimize the predictive accuracy of the neural network.

 * Number of hidden layers (1, 2, 3)
 * Size of hidden layers
 * 2 or 3 output categories
 * Dropout rate (0.1 in some papers)
 * Optimization algorithm (Adaptive Moment, Gradient Descent)
 * Learning rate
 
Measures of predictive accuracy (see [Receiver Operating Characteristic](https://en.wikipedia.org/wiki/Receiver_operating_characteristic)):
 * Accuracy: percentage of correct predictions
 * Area Under the Curve: the area under the curve parameterized by a cut-off between binary classifications on the axes of true positive rate over false positive rate

In [18]:
default_nn_config = {
    "df": df_training,
    "df_validation": df_valid,
    "layer_sizes": (70, 20),
    "dropout": 0.1,
    "batch_size": 64
}

param_choices = {
    "dropout": [0.0, 0.5, 0.1, 0.15, 0.2],
    "batch_size": [32, 64, 128, 256],
    "layer_sizes": [(60, 20), (70, 20), (80, 20), (70, 10), (70, 20), (70, 30), (70,), (100,), (70, 20, 10)]
}

configs = []
for output_categories in ["three", "any", "rapid"]:
    for param, choices in param_choices.items():
        for choice in choices:
            config = default_nn_config.copy()
            config["output_categories"] = output_categories
            config[param] = choice
            configs.append(config)
            
def configStr(config):
    return "Output Categories: {}; Layer Sizes: {}; Dropout: {}; Batch Size: {}".format(
        config["output_categories"], config["layer_sizes"], config["dropout"], config["batch_size"])

In [19]:
history = {}
for config in configs:
    subhist = []
    for i in range(5):
        nn = NeuralNetwork(**config)
        subhist.append(nn.train(20))
    history[configStr(config)] = subhist

Train on 110208 samples, validate on 14901 samples
Epoch 1/20
110208/110208 [==============================] - 4s 37us/sample - loss: 0.6284 - acc: 0.6678 - auc: 0.5703 - val_loss: 0.6259 - val_acc: 0.6675 - val_auc: 0.5898
Epoch 2/20
110208/110208 [==============================] - 4s 36us/sample - loss: 0.6155 - acc: 0.6730 - auc: 0.6184 - val_loss: 0.6286 - val_acc: 0.6584 - val_auc: 0.5971
Epoch 3/20
110208/110208 [==============================] - 4s 36us/sample - loss: 0.6122 - acc: 0.6741 - auc: 0.6272 - val_loss: 0.6126 - val_acc: 0.6727 - val_auc: 0.6401
Epoch 4/20
110208/110208 [==============================] - 4s 36us/sample - loss: 0.6115 - acc: 0.6747 - auc: 0.6292 - val_loss: 0.6074 - val_acc: 0.6797 - val_auc: 0.6489
Epoch 5/20
110208/110208 [==============================] - 4s 36us/sample - loss: 0.6111 - acc: 0.6744 - auc: 0.6302 - val_loss: 0.6068 - val_acc: 0.6801 - val_auc: 0.6496
Epoch 6/20
110208/110208 [==============================] - 4s 36us/sample - loss: 0

110208/110208 [==============================] - 4s 37us/sample - loss: 0.6103 - acc: 0.6754 - auc: 0.6319 - val_loss: 0.5900 - val_acc: 0.6898 - val_auc: 0.6872
Epoch 8/20
110208/110208 [==============================] - 4s 37us/sample - loss: 0.6100 - acc: 0.6754 - auc: 0.6325 - val_loss: 0.6179 - val_acc: 0.6696 - val_auc: 0.6208
Epoch 9/20
110208/110208 [==============================] - 4s 37us/sample - loss: 0.6096 - acc: 0.6757 - auc: 0.6334 - val_loss: 0.5985 - val_acc: 0.6838 - val_auc: 0.6683
Epoch 10/20
110208/110208 [==============================] - 4s 38us/sample - loss: 0.6094 - acc: 0.6761 - auc: 0.6339 - val_loss: 0.6023 - val_acc: 0.6819 - val_auc: 0.6604
Epoch 11/20
110208/110208 [==============================] - 4s 38us/sample - loss: 0.6090 - acc: 0.6763 - auc: 0.6346 - val_loss: 0.6183 - val_acc: 0.6713 - val_auc: 0.6158
Epoch 12/20
110208/110208 [==============================] - 4s 37us/sample - loss: 0.6089 - acc: 0.6760 - auc: 0.6353 - val_loss: 0.6011 - val_

Epoch 14/20
110208/110208 [==============================] - 4s 37us/sample - loss: 0.6085 - acc: 0.6765 - auc: 0.6362 - val_loss: 0.6033 - val_acc: 0.6800 - val_auc: 0.6569
Epoch 15/20
110208/110208 [==============================] - 4s 38us/sample - loss: 0.6083 - acc: 0.6769 - auc: 0.6364 - val_loss: 0.5964 - val_acc: 0.6866 - val_auc: 0.6721
Epoch 16/20
110208/110208 [==============================] - 4s 38us/sample - loss: 0.6080 - acc: 0.6769 - auc: 0.6374 - val_loss: 0.6162 - val_acc: 0.6721 - val_auc: 0.6240
Epoch 17/20
110208/110208 [==============================] - 4s 38us/sample - loss: 0.6079 - acc: 0.6772 - auc: 0.6376 - val_loss: 0.6140 - val_acc: 0.6748 - val_auc: 0.6303
Epoch 18/20
110208/110208 [==============================] - 4s 38us/sample - loss: 0.6078 - acc: 0.6773 - auc: 0.6380 - val_loss: 0.6018 - val_acc: 0.6820 - val_auc: 0.6613
Epoch 19/20
110208/110208 [==============================] - 4s 38us/sample - loss: 0.6075 - acc: 0.6776 - auc: 0.6385 - val_loss:

110208/110208 [==============================] - 4s 41us/sample - loss: 0.6146 - acc: 0.6733 - auc: 0.6207 - val_loss: 0.6114 - val_acc: 0.6730 - val_auc: 0.6414
Train on 110208 samples, validate on 14901 samples
Epoch 1/20
110208/110208 [==============================] - 5s 46us/sample - loss: 0.6360 - acc: 0.6667 - auc: 0.5153 - val_loss: 0.6262 - val_acc: 0.6667 - val_auc: 0.6059
Epoch 2/20
110208/110208 [==============================] - 5s 43us/sample - loss: 0.6280 - acc: 0.6673 - auc: 0.5758 - val_loss: 0.6285 - val_acc: 0.6703 - val_auc: 0.5662
Epoch 3/20
110208/110208 [==============================] - 5s 42us/sample - loss: 0.6245 - acc: 0.6687 - auc: 0.5891 - val_loss: 0.6249 - val_acc: 0.6684 - val_auc: 0.5853
Epoch 4/20
110208/110208 [==============================] - 5s 43us/sample - loss: 0.6218 - acc: 0.6702 - auc: 0.5976 - val_loss: 0.6258 - val_acc: 0.6650 - val_auc: 0.5852
Epoch 5/20
110208/110208 [==============================] - 4s 41us/sample - loss: 0.6204 - acc

110208/110208 [==============================] - 5s 46us/sample - loss: 0.6189 - acc: 0.6720 - auc: 0.6057 - val_loss: 0.6164 - val_acc: 0.6683 - val_auc: 0.6336
Epoch 7/20
110208/110208 [==============================] - 5s 46us/sample - loss: 0.6189 - acc: 0.6720 - auc: 0.6056 - val_loss: 0.6136 - val_acc: 0.6713 - val_auc: 0.6390
Epoch 8/20
110208/110208 [==============================] - 5s 46us/sample - loss: 0.6180 - acc: 0.6727 - auc: 0.6083 - val_loss: 0.6055 - val_acc: 0.6749 - val_auc: 0.6634
Epoch 9/20
110208/110208 [==============================] - 5s 46us/sample - loss: 0.6176 - acc: 0.6727 - auc: 0.6088 - val_loss: 0.6104 - val_acc: 0.6720 - val_auc: 0.6526
Epoch 10/20
110208/110208 [==============================] - 5s 46us/sample - loss: 0.6175 - acc: 0.6725 - auc: 0.6102 - val_loss: 0.6109 - val_acc: 0.6728 - val_auc: 0.6439
Epoch 11/20
110208/110208 [==============================] - 5s 46us/sample - loss: 0.6169 - acc: 0.6730 - auc: 0.6119 - val_loss: 0.6113 - val_a

...

123200/123200 [==============================] - 19s 152us/sample - loss: 0.6511 - acc: 0.6143 - auc: 0.6605 - val_loss: 0.6706 - val_acc: 0.5773 - val_auc: 0.6255
Epoch 20/20
123200/123200 [==============================] - 19s 152us/sample - loss: 0.6506 - acc: 0.6139 - auc: 0.6609 - val_loss: 0.6829 - val_acc: 0.5641 - val_auc: 0.6015
Train on 123200 samples, validate on 14901 samples
Epoch 1/20
123200/123200 [==============================] - 31s 255us/sample - loss: 0.6755 - acc: 0.5746 - auc: 0.6051 - val_loss: 0.6634 - val_acc: 0.6139 - val_auc: 0.6500
Epoch 2/20
123200/123200 [==============================] - 19s 154us/sample - loss: 0.6639 - acc: 0.5947 - auc: 0.6343 - val_loss: 0.6915 - val_acc: 0.5814 - val_auc: 0.5951
Epoch 3/20
123200/123200 [==============================] - 19s 155us/sample - loss: 0.6618 - acc: 0.5983 - auc: 0.6390 - val_loss: 0.6473 - val_acc: 0.6477 - val_auc: 0.6975
Epoch 4/20
123200/123200 [==============================] - 19s 152us/sample - loss:

Epoch 5/20
123200/123200 [==============================] - 19s 154us/sample - loss: 0.6596 - acc: 0.6026 - auc: 0.6433 - val_loss: 0.6425 - val_acc: 0.6450 - val_auc: 0.7045
Epoch 6/20
123200/123200 [==============================] - 19s 153us/sample - loss: 0.6586 - acc: 0.6028 - auc: 0.6453 - val_loss: 0.6870 - val_acc: 0.5724 - val_auc: 0.6022
Epoch 7/20
123200/123200 [==============================] - 19s 153us/sample - loss: 0.6579 - acc: 0.6033 - auc: 0.6466 - val_loss: 0.6836 - val_acc: 0.5739 - val_auc: 0.6111
Epoch 8/20
123200/123200 [==============================] - 19s 156us/sample - loss: 0.6572 - acc: 0.6068 - auc: 0.6486 - val_loss: 0.6291 - val_acc: 0.6537 - val_auc: 0.7226
Epoch 9/20
123200/123200 [==============================] - 19s 157us/sample - loss: 0.6566 - acc: 0.6049 - auc: 0.6496 - val_loss: 0.6793 - val_acc: 0.5730 - val_auc: 0.6153
Epoch 10/20
123200/123200 [==============================] - 19s 154us/sample - loss: 0.6561 - acc: 0.6058 - auc: 0.6504 - va

123200/123200 [==============================] - 19s 156us/sample - loss: 0.6554 - acc: 0.6074 - auc: 0.6519 - val_loss: 0.6666 - val_acc: 0.5898 - val_auc: 0.6402
Epoch 12/20
123200/123200 [==============================] - 19s 157us/sample - loss: 0.6547 - acc: 0.6089 - auc: 0.6533 - val_loss: 0.6705 - val_acc: 0.5845 - val_auc: 0.6325
Epoch 13/20
123200/123200 [==============================] - 19s 158us/sample - loss: 0.6541 - acc: 0.6094 - auc: 0.6546 - val_loss: 0.6561 - val_acc: 0.6063 - val_auc: 0.6623
Epoch 14/20
123200/123200 [==============================] - 19s 157us/sample - loss: 0.6536 - acc: 0.6099 - auc: 0.6557 - val_loss: 0.6539 - val_acc: 0.6039 - val_auc: 0.6630
Epoch 15/20
123200/123200 [==============================] - 19s 155us/sample - loss: 0.6530 - acc: 0.6113 - auc: 0.6571 - val_loss: 0.6667 - val_acc: 0.5832 - val_auc: 0.6358
Epoch 16/20
123200/123200 [==============================] - 19s 156us/sample - loss: 0.6525 - acc: 0.6122 - auc: 0.6580 - val_loss:

123200/123200 [==============================] - 20s 164us/sample - loss: 0.6548 - acc: 0.6090 - auc: 0.6543 - val_loss: 0.6389 - val_acc: 0.6106 - val_auc: 0.6904
Epoch 18/20
123200/123200 [==============================] - 20s 164us/sample - loss: 0.6546 - acc: 0.6092 - auc: 0.6547 - val_loss: 0.6406 - val_acc: 0.6169 - val_auc: 0.6893
Epoch 19/20
123200/123200 [==============================] - 20s 161us/sample - loss: 0.6537 - acc: 0.6114 - auc: 0.6565 - val_loss: 0.6822 - val_acc: 0.5477 - val_auc: 0.6012
Epoch 20/20
123200/123200 [==============================] - 20s 162us/sample - loss: 0.6535 - acc: 0.6109 - auc: 0.6567 - val_loss: 0.6319 - val_acc: 0.6061 - val_auc: 0.6933
Train on 123200 samples, validate on 14901 samples
Epoch 1/20
123200/123200 [==============================] - 33s 265us/sample - loss: 0.6843 - acc: 0.5452 - auc: 0.5720 - val_loss: 0.6717 - val_acc: 0.5582 - val_auc: 0.6272
Epoch 2/20
123200/123200 [==============================] - 20s 161us/sample - los

Epoch 3/20
123200/123200 [==============================] - 20s 166us/sample - loss: 0.6640 - acc: 0.5994 - auc: 0.6365 - val_loss: 0.6637 - val_acc: 0.6094 - val_auc: 0.6483
Epoch 4/20
123200/123200 [==============================] - 20s 165us/sample - loss: 0.6615 - acc: 0.6028 - auc: 0.6416 - val_loss: 0.6496 - val_acc: 0.6267 - val_auc: 0.6778
Epoch 5/20
123200/123200 [==============================] - 20s 164us/sample - loss: 0.6607 - acc: 0.6021 - auc: 0.6428 - val_loss: 0.6882 - val_acc: 0.5730 - val_auc: 0.6037
Epoch 6/20
123200/123200 [==============================] - 20s 164us/sample - loss: 0.6598 - acc: 0.6042 - auc: 0.6446 - val_loss: 0.6361 - val_acc: 0.6339 - val_auc: 0.7023
Epoch 7/20
123200/123200 [==============================] - 20s 165us/sample - loss: 0.6593 - acc: 0.6037 - auc: 0.6454 - val_loss: 0.6471 - val_acc: 0.5987 - val_auc: 0.6747
Epoch 8/20
123200/123200 [==============================] - 20s 166us/sample - loss: 0.6584 - acc: 0.6041 - auc: 0.6468 - val

In [26]:
import json
with open("training_history.json", "w") as outfile:
    json.dump({key: [str(i.history) for i in value] for key, value in history.items()}, outfile)